Thinking1：如何使用用户标签来指导业务（如何提升业务）

A1：利用用户标签分析用户属性、用户习惯等来达到

获客：如何进行拉新，通过更精准的营销获取客户

粘客：个性化推荐，排序搜索，场景运营等

留客：流失率预测，分析关键节点降低流失率


Thinking2：如果给你一堆用户数据，没有打标签。你该如何处理（如何打标签）	

A2：

1.统一标识，唯一的userid

2.用户标签：性别、年龄、地域、收入、学历、职业等

3.消费标签：消费习惯、购买意向、是否对促销感兴趣等
   
4.行为标签：评分、喜欢、点击、收藏、浏览时长等

5.内容分析：对用户平时浏览的内容进行分析

Thinking3：准确率和精确率有何不同（评估指标）

A3：

1）True positives(TP):  被正确地划分为正例的个数

2）False positives(FP): 被错误地划分为正例的个数

3）False negatives(FN):被错误地划分为负例的个数

4）True negatives(TN): 被正确地划分为负例的个数

accuracy = （TP+TN）/(P+N)

precision=TP/（TP+FP）

分类正确率，不管是哪个类别，只要预测正确，其数量都放在分子上，而分母是全部数据数量，这说明正确率是对全部数据的判断。而准确率在分类中对应的是某个类别，分子是预测该类别正确的数量，分母是预测为该类别的全部数据的数量。或者说，Accuracy是对分类器整体上的正确率的评价，而Precision是分类器预测为某一个类别的正确率的评价。


Thinking4：如果你使用大众点评，想要给某个餐厅打标签。这时系统可以自动提示一些标签，你会如何设计（标签推荐）				

A4：
用户u给物品i打标签，自动提示可以有

方法一：给用户u自动推荐整个系统最热门的标签/不推荐使用，因为范维太广，不适合做个性化的推荐

方法二：给用户u推荐物品i上最热门的标签

方法三：给用户u推荐他自己经常使用的标签

or将方法2和方法3进行加权融合，生成最终的标签结果



Thinking5：我们今天使用了10种方式来解MNIST，这些方法有何不同？你还有其他方法来解决MNIST识别问题么（分类方法）				

A5:

有监督学习的方法，也有非监督学习的方法。有类似于AutoML以及深度学习的框架

还可以使用Ensemble Learning的思想，将多个模型相融合，提高正确率

In [9]:
import random
import math
import operator
 
class NormTagBased():
    # 构造函数
    def __init__(self,filename):
        self.filename=filename
        self.loadData()
        self.randomlySplitData(0.2)
        self.initStat()
        self.testRecommend()
        
    # 数据加载
    def loadData(self):
        print("开始数据加载...")
        filename=self.filename
        # 保存了用户对item的tag
        self.records={}
        fi=open(filename)
        lineNum=0
        for line in fi:
            lineNum+=1
            if lineNum==1:
                continue
            uid,iid,tag,timestamp=line.split('\t')
            # 在数组中对应的下标-1
            uid=int(uid)-1
            iid=int(iid)-1
            tag=int(tag)-1
            self.records.setdefault(uid,{})
            self.records[uid].setdefault(iid,[])
            self.records[uid][iid].append(tag)
        fi.close()
        print("数据集大小为 %d." % (lineNum))
        print("设置tag的人数 %d." % (len(self.records)))
        print("数据加载完成\n")
    
    # 将数据集拆分为训练集和测试集
    def randomlySplitData(self,ratio,seed=100):
        random.seed(seed)
        self.train=dict()
        self.test=dict()
        for u in self.records.keys():
            for i in self.records[u].keys():
                if random.random()<ratio:
                    self.test.setdefault(u,{})
                    self.test[u].setdefault(i,[])
                    for t in self.records[u][i]:
                        self.test[u][i].append(t)
                else:
                    self.train.setdefault(u,{})
                    self.train[u].setdefault(i,[])
                    for t in self.records[u][i]:
                        self.train[u][i].append(t)        
        print("训练集样本数 %d, 测试集样本数 %d" % (len(self.train),len(self.test)))
    
    # 使用训练集，初始化user_tags, tag_items, user_items
    # user_tags[u][t]用户打过的标签
    # user_items[u][i] 用户打过标签的商品
    # tag_items[t][i] 打上某标签的商品
    def initStat(self):
        records=self.train
        self.user_tags=dict()
        self.tag_items=dict()
        self.user_items=dict()
        self.tag_users=dict()
        for u,items in records.items():
            for i,tags in items.items():
                for tag in tags:
                    #print tag
                    # 用户和tag的关系
                    self._addValueToMat(self.user_tags,u,tag,1)
                    # tag和item的关系
                    self._addValueToMat(self.tag_items,tag,i,1)
                    # 用户和item的关系
                    self._addValueToMat(self.user_items,u,i,1)
                    # tag和用户的关系
                    self._addValueToMat(self.tag_users,tag,u,1)
        print("user_tags, tag_items, user_items初始化完成.")
        print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d, tag_users大小 %d" % (len(self.user_tags),len(self.tag_items),len(self.user_items),len(self.tag_users)))
    
    # 设置矩阵 mat[index, item] = 1
    def _addValueToMat(self,mat,index,item,value=1):
        if index not in mat:
            mat.setdefault(index,{})
            mat[index].setdefault(item,value)
        else:
            if item not in mat[index]:
                mat[index][item] = value
            else:
                mat[index][item] += value
    
    # 使用测试集，计算准确率和召回率
    def precisionAndRecall(self, N):
        hit=0
        h_recall=0
        h_precision=0
        for user,items in self.test.items():
            if user not in self.train:
                continue
            # 获取Top-N推荐列表
            rank=self.recommend(user,N)
            for item,rui in rank:
                if item in items:
                    hit+=1
            h_recall+=len(items)
            h_precision+=N
        #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
        # 返回准确率 和 召回率
        return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))
    
    # 对用户user推荐Top-N
    def recommend(self,user,N):
        recommend_items=dict()
        # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
        tagged_items = self.user_items[user]     
        for tag, wut in self.user_tags[user].items():
            #print(self.user_tags[user].items())
            for item, wti in self.tag_items[tag].items():
                if item in tagged_items:
                    continue
                #print('wut = %s, wti = %s' %(wut, wti))
                if item not in recommend_items:
                    recommend_items[item] = (wut/ len(self.user_tags[user])) * (wti/len(self.tag_users[tag]))
                else:
                    recommend_items[item] += (wut/ len(self.user_tags[user])) * (wti/len(self.tag_users[tag]))
        return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]
    
    # 使用测试集，对推荐结果进行评估
    def testRecommend(self):
        print("推荐结果评估")
        print("%3s %10s %10s" % ('N',"精确率",'召回率'))
        for n in [5,10,20,40,60,80,100]:
            precision,recall=self.precisionAndRecall(n)
            print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))
        
if __name__=='__main__':
    stb=NormTagBased("./user_taggedbookmarks-timestamps.dat")

开始数据加载...
数据集大小为 437594.
设置tag的人数 1867.
数据加载完成

训练集样本数 1860, 测试集样本数 1793
user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860, tag_users大小 36884
推荐结果评估
  N        精确率        召回率
  5      0.907%      0.388%
 10      0.638%      0.546%
 20      0.507%      0.868%
 40      0.356%      1.218%
 60      0.287%      1.476%
 80      0.255%      1.750%
100      0.241%      2.061%


In [11]:
import random
import math
import operator
 
class TagBasedTFIDF():
    # 构造函数
    def __init__(self,filename):
        self.filename=filename
        self.loadData()
        self.randomlySplitData(0.2)
        self.initStat()
        self.testRecommend()
        
    # 数据加载
    def loadData(self):
        print("开始数据加载...")
        filename=self.filename
        # 保存了用户对item的tag
        self.records={}
        fi=open(filename)
        lineNum=0
        for line in fi:
            lineNum+=1
            if lineNum==1:
                continue
            uid,iid,tag,timestamp=line.split('\t')
            # 在数组中对应的下标-1
            uid=int(uid)-1
            iid=int(iid)-1
            tag=int(tag)-1
            self.records.setdefault(uid,{})
            self.records[uid].setdefault(iid,[])
            self.records[uid][iid].append(tag)
        fi.close()
        print("数据集大小为 %d." % (lineNum))
        print("设置tag的人数 %d." % (len(self.records)))
        print("数据加载完成\n")
    
    # 将数据集拆分为训练集和测试集
    def randomlySplitData(self,ratio,seed=100):
        random.seed(seed)
        self.train=dict()
        self.test=dict()
        for u in self.records.keys():
            for i in self.records[u].keys():
                if random.random()<ratio:
                    self.test.setdefault(u,{})
                    self.test[u].setdefault(i,[])
                    for t in self.records[u][i]:
                        self.test[u][i].append(t)
                else:
                    self.train.setdefault(u,{})
                    self.train[u].setdefault(i,[])
                    for t in self.records[u][i]:
                        self.train[u][i].append(t)        
        print("训练集样本数 %d, 测试集样本数 %d" % (len(self.train),len(self.test)))
    
    # 使用训练集，初始化user_tags, tag_items, user_items
    # user_tags[u][t]用户打过的标签
    # user_items[u][i] 用户打过标签的商品
    # tag_items[t][i] 打上某标签的商品
    def initStat(self):
        records=self.train
        self.user_tags=dict()
        self.tag_items=dict()
        self.user_items=dict()
        self.tag_users=dict()
        for u,items in records.items():
            for i,tags in items.items():
                for tag in tags:
                    #print tag
                    # 用户和tag的关系
                    self._addValueToMat(self.user_tags,u,tag,1)
                    # tag和item的关系
                    self._addValueToMat(self.tag_items,tag,i,1)
                    # 用户和item的关系
                    self._addValueToMat(self.user_items,u,i,1)
                    # tag和用户的关系
                    self._addValueToMat(self.tag_users,tag,u,1)
        print("user_tags, tag_items, user_items初始化完成.")
        print("user_tags大小 %d, tag_items大小 %d, user_items大小 %d, tag_users大小 %d" % (len(self.user_tags),len(self.tag_items),len(self.user_items),len(self.tag_users)))
    
    # 设置矩阵 mat[index, item] = 1
    def _addValueToMat(self,mat,index,item,value=1):
        if index not in mat:
            mat.setdefault(index,{})
            mat[index].setdefault(item,value)
        else:
            if item not in mat[index]:
                mat[index][item] = value
            else:
                mat[index][item] += value
    
    # 使用测试集，计算准确率和召回率
    def precisionAndRecall(self, N):
        hit=0
        h_recall=0
        h_precision=0
        for user,items in self.test.items():
            if user not in self.train:
                continue
            # 获取Top-N推荐列表
            rank=self.recommend(user,N)
            for item,rui in rank:
                if item in items:
                    hit+=1
            h_recall+=len(items)
            h_precision+=N
        #print('一共命中 %d 个, 一共推荐 %d 个, 用户设置tag总数 %d 个' %(hit, h_precision, h_recall))
        # 返回准确率 和 召回率
        return (hit/(h_precision*1.0)), (hit/(h_recall*1.0))
    
    # 对用户user推荐Top-N
    def recommend(self,user,N):
        recommend_items=dict()
        # 对Item进行打分，分数为所有的（用户对某标签使用的次数 wut, 乘以 商品被打上相同标签的次数 wti）之和
        tagged_items = self.user_items[user]     
        for tag, wut in self.user_tags[user].items():
            #print(self.user_tags[user].items())
            for item, wti in self.tag_items[tag].items():
                if item in tagged_items:
                    continue
                #print('wut = %s, wti = %s' %(wut, wti))
                if item not in recommend_items:
                    recommend_items[item] = (wut/ math.log(len(self.tag_users[tag])+1)) * wti
                else:
                    recommend_items[item] += (wut/ math.log(len(self.tag_users[tag])+1)) * wti
        return sorted(recommend_items.items(), key=operator.itemgetter(1), reverse=True)[0:N]
    
    # 使用测试集，对推荐结果进行评估
    def testRecommend(self):
        print("推荐结果评估")
        print("%3s %10s %10s" % ('N',"精确率",'召回率'))
        for n in [5,10,20,40,60,80,100]:
            precision,recall=self.precisionAndRecall(n)
            print("%3d %10.3f%% %10.3f%%" % (n, precision * 100, recall * 100))
        
if __name__=='__main__':
    stb=TagBasedTFIDF("./user_taggedbookmarks-timestamps.dat")

开始数据加载...
数据集大小为 437594.
设置tag的人数 1867.
数据加载完成

训练集样本数 1860, 测试集样本数 1793
user_tags, tag_items, user_items初始化完成.
user_tags大小 1860, tag_items大小 36884, user_items大小 1860, tag_users大小 36884
推荐结果评估
  N        精确率        召回率
  5      1.008%      0.431%
 10      0.761%      0.652%
 20      0.549%      0.940%
 40      0.402%      1.376%
 60      0.328%      1.687%
 80      0.297%      2.033%
100      0.269%      2.306%


kaggle链接

https://www.kaggle.com/zhangwx95/most-vote-study-use-rf